## Running ISR from a notebook

Craig Lage - 16-Jun-22

    
        Here's an example of running ISR with a pipetask using the yaml file:

          isr:
            class: lsst.ip.isr.IsrTask
            config:
              connections.ccdExposure: raw
              connections.outputExposure: parameters.exposureName
              doWrite: true
              doOverscan: true
              doAssembleCcd: true
              doBias: true
              doVariance: true
              doLinearize: false
              doCrosstalk: false
              doBrighterFatter: false
              doDark: true
              doStrayLight: false
              doFlat: false
              doFringe: false
              doApplyGains: false
              doDefect: true
              doNanMasking: true
              doInterpolate: false
              doSaturation: false
              doSaturationInterpolation: false
              growSaturationFootprintSize: 0
     

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.ip.isr import IsrTask, IsrTaskConfig
from lsst.daf.butler import Butler

In [ ]:
butler = Butler('/repo/LATISS', collections=["LATISS/raw/all", "LATISS/calib"])

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.overscan.fitType='MEDIAN_PER_ROW'
isrConfig.overscan.doParallelOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=False
isrConfig.doVariance=True
isrConfig.doLinearize=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=False
isrConfig.doDefect=False
isrConfig.doNanMasking=True
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False


# Adjust these as needed and add as many more as you want

In [ ]:
isrTask = IsrTask(config=isrConfig)

In [ ]:
expId = 2023062900056
exp = butler.get('raw', detector=0, exposure=expId)

In [ ]:
isrResult = isrTask.run(exp)

In [ ]:
# Now look at the data with matplotlib
from matplotlib.colors import LogNorm

def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(8,8))
plt.suptitle(f"Image",fontsize=18)
arr = isrResult.exposure.image.array[2750:3250,250:750]
img = plt.imshow(arr, vmin=1, vmax=10, interpolation='Nearest', cmap='gray')
colorbar(img)
plt.tight_layout(h_pad=1)
plt.savefig("/home/craiglagegit/DATA/Dark_2023062900056.png")